In [1]:
import pandas as pd
import os

In [2]:
def _to_float(s):
    return float(s[:-2] + '.' + s[-2:])

In [3]:
# Lista de arquivos Parquet na camada bronze
file_list = [
    "COTAHIST_A2019",
    "COTAHIST_A2020",
    "COTAHIST_A2021",
    "COTAHIST_A2022",
    "COTAHIST_A2023",
    "COTAHIST_A2024",
]

# Garantir que a pasta silver exista
os.makedirs("silver/tickers_bolsa", exist_ok=True)

# Loop pelos arquivos
for file in file_list:
    # Caminho do arquivo parquet da camada bronze
    bronze_path = f"bronze/tickers_bolsa/{file}.parquet"
    
    # Carregar os dados da camada bronze
    df = pd.read_parquet(bronze_path)
    df
    
    # Conversão de tipos
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='coerce')
    df['close'] = df['close'].apply(_to_float)
    df['ticker'] = df['ticker'].astype(str).str.upper().str.strip()
    
    # Remover linhas com dados nulos
    df = df.dropna(subset=['date', 'ticker', 'close'])
    
    # Remover duplicatas
    df = df.drop_duplicates(subset=['date', 'ticker'])
    
    # Ordenar os dados
    df = df.sort_values(by=['ticker', 'date']).reset_index(drop=True)
    
    # Caminho da camada silver
    silver_path = f"silver/tickers_bolsa/{file}.parquet"
    
    # Salvar arquivo limpo na camada silver
    df.to_parquet(silver_path, index=False)

    print(f"{file} processado e salvo em silver.")
df

COTAHIST_A2019 processado e salvo em silver.
COTAHIST_A2020 processado e salvo em silver.
COTAHIST_A2021 processado e salvo em silver.
COTAHIST_A2022 processado e salvo em silver.
COTAHIST_A2023 processado e salvo em silver.
COTAHIST_A2024 processado e salvo em silver.


,date,ticker,close
0,2024-01-02,5GTK11,81.39
1,2024-01-03,5GTK11,80.10
2,2024-01-04,5GTK11,79.47
3,2024-01-05,5GTK11,79.34
4,2024-01-08,5GTK11,81.47
...,...,...,...
2607843,2024-12-16,ZIFI11,610.00
2607844,2024-12-20,ZIFI11,601.14
2607845,2024-12-23,ZIFI11,603.02
2607846,2024-12-26,ZIFI11,620.00
